In [ ]:
#| default_exp classes.DomoCard

In [ ]:
#| exporti
from dataclasses import dataclass, field

import asyncio
from fastcore.basics import patch_to

import domolibrary.routes.card as card_routes

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.utils.DictDot as util_dd

In [ ]:
# | export
@dataclass
class DomoCard:
    id: str
    auth: dmda.DomoAuth = field(repr=False)
    title: str = None
    description: str = None
    type: str = None
    urn: str = None
    chart_type: str = None
    dataset_id: str = None

    certification: field(default_factory=dict) = None
    owners: field(default_factory=list) = None

    def __post_init__(self):
        # self.Definition = CardDefinition(self)
        pass

    def display_url(self) -> str:
        return f'https://{self.domo_instance}.domo.com/kpis/details/{self.id}'

    @classmethod
    async def _from_json(cls,
                  card_obj,
                   auth: dmda.DomoAuth):

        import domolibrary.classes.DomoUser as dmu
        import domolibrary.classes.DomoGroup as dmg

        dd = card_obj
        if isinstance(card_obj, dict):
            dd = util_dd.DictDot(card_obj)

        card = cls(
            auth=auth,
            id=dd.id,
            title=dd.title,
            description=dd.description,
            type=dd.type,
            urn=dd.urn,
            certification=dd.certification,
            chart_type=dd.metadata.chartType,
            dataset_id = dd.datasources[0].dataSourceId if dd.datasources else None
        )

        tasks = []
        for user in dd.owners:
            if user.type =='USER':
                tasks.append(dmu.DomoUser.get_by_id(user_id=user.id, auth=auth))
            if user.type == 'GROUP':
                tasks.append(dmg.DomoGroup.get_by_id(group_id=user.id, auth=auth))

        card.owners = await asyncio.gather( *tasks)

        return card


In [ ]:
@patch_to(DomoCard, cls_method=True)
async def get_by_id(cls: DomoCard, card_id: str,
                      auth: dmda.DomoAuth, debug_api: bool = False):

    res = await card_routes.get_card_metadata(auth=auth,
                                              card_id=card_id, debug_api=debug_api)

    if not res.is_success:
        raise Exception('unable to retrieve card {card_id}')

    domo_card = await cls._from_json(res.response, auth)

    return domo_card


In [ ]:
import os
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoCard.get_by_id(card_id=1766265020, auth=token_auth)

DomoCard(id=1766265020, title='Vendor Amounts Dates', description='', type='kpi', urn='1766265020', chart_type='badge_vert_stackedbar', dataset_id='fd70d83b-38dc-40a2-9349-357ec4321d3e', certification=DictDot(state='NOT_CERTIFIED', adminCertified=False), owners=[DomoUser(id='1893952720', title=None, department='Business Improvement', display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=1, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)])

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()

SyntaxError: invalid syntax (<unknown>, line 2)

Bad pipe message: %s [b"\xcb\x81vP\x8d|\xdd\xdf\x00\\\xf4;3,\xc3\xf2\x80w T\xe0\t\x9e\xc7FD_\xb4jiu2\xc7\x1e9\x8c\xad\xbd\xd3\xbe7\xf1\xed'fq<\xe8\x03\xa7\x8b\x00\x08\x13\x02\x13\x03\x13", b'']
Bad pipe message: %s [b'\xb8\xbe\xa3$XO\x04\xf3"\x19\x19\xb3=\x11\x90\xdb\xc1\xa4 \x17\xdf"]\x8ee\x98HI\xaf\xb2\x81\x8a\x8a\xde\xa1']
Bad pipe message: %s [b'e\x935\xfb\xe8\xa9\xb0\xe2\x17\xaaa]\xfdA\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.']
Bad pipe message: %s [b'1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03']
Bad pipe message: %s [b'\x08\x07\x08\x08\x08', b'\n\x08\x0b\x08\x04\x08\x05\x08']
Bad pipe message: %s [b'\x01\x05\x01\x06\x01']
Bad pipe message: %s [b':\xc0-\xd4\x02\xe7%\xdc\x896\t\xb2\xbd,\x9cwM6\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00:\x00\x89\xc0\x0